
# 문제 1-3 : FewShotPromptTemplate — 뉴스 키워드 추출기

**목표**:  
- `FewShotChatMessagePromptTemplate`로 예시 3개 이상 제공  
- 시스템 메시지를 통해 **정확히 3개 키워드**만, `키워드: A, B, C` 형식으로 출력  


In [1]:

# 필수 패키지(처음 1회만)
# !pip install -U langchain-core langchain-openai python-dotenv


In [2]:

import os
from dotenv import load_dotenv, find_dotenv
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

env_path = find_dotenv(usecwd=True)
load_dotenv(env_path)

API_KEY = os.getenv("OPENAI_API_KEY") or os.getenv("GROQ_API_KEY")
if not API_KEY:
    raise RuntimeError(f".env에 OPENAI_API_KEY 또는 GROQ_API_KEY가 없습니다. 찾은 경로: {env_path or '(없음)'}")

print("[API prefix]", API_KEY[:6])

llm = ChatOpenAI(
    api_key=API_KEY,
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.2,  # 키워드 추출은 일관성이 좋아야 해서 낮춤
)


[API prefix] gsk_Zl


In [3]:

# 예시 프롬프트(대화형)
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{news}"),
    ("ai", "키워드: {keywords}")
])

# 최소 3개 예시
examples = [
    {
        "news": "삼성전자가 내년 초 자체 개발한 인공지능(AI) 가속기를 출시할 예정이다. 엔비디아의 독점 구조에 도전한다.",
        "keywords": "삼성전자, AI가속기, 엔비디아",
    },
    {
        "news": "세계보건기구(WHO)는 새로운 건강 위기에 대응하기 위해 국제 협력의 중요성을 강조하며, 글로벌 보건 시스템 개선을 촉구했다.",
        "keywords": "WHO, 국제협력, 보건시스템",
    },
    {
        "news": "테슬라는 차세대 배터리 생산 공정을 공개하며 생산 단가 절감을 목표로 한다고 밝혔다.",
        "keywords": "테슬라, 배터리, 생산공정",
    },
]

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "뉴스 키워드 추출 전문가입니다. 핵심 키워드 3개만, 쉼표로 구분해 '키워드: A, B, C' 형식으로 출력하세요."),
    few_shot_prompt,
    ("human", "{input}")
])

chain = final_prompt | llm | StrOutputParser()


In [4]:
test_news = (
    "제미나이 2.0 플래시는 현재 구글 AI 스튜디오(Google AI Studio) 및 버텍스 AI(Vertex AI)에서 "
    "제미나이 API를 통해 개발자에게 실험 모델로 제공됩니다. 멀티모달 입력 및 텍스트 출력을 사용할 수 있으며, "
    "텍스트 음성 변환과 네이티브 이미지 생성은 일부 파트너들을 대상으로 제공됩니다. 내년 1월에는 "
    "더 많은 모델 사이즈와 함께 일반에 공개될 예정입니다."
)

print(chain.invoke({"input": test_news}))


키워드: 제미나이, AI, 구글
